# 2.7 모델 세부 튜닝

## 2.7.0 이전 내용 복습

In [1]:
##############################################################################

## 데이터 추출 함수

import os
import tarfile
import urllib

DOWNLOAD_ROOT = 'https://raw.githubusercontent.com/ageron/handson-ml2/master/'
HOUSING_PATH = '../' + os.path.join('datasets', 'housing')
HOUSING_URL = DOWNLOAD_ROOT + 'datasets/housing/housing.tgz'

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    """
    현재 작업 공간에 "datasets/housing" 디렉터리 생성
    housing.tgz 파일 내려받음
    같인 디렉터리에 압축을 풀어 "housing.csv" 파일 생성
    """
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, 'housing.tgz')
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
    
#fetch_housing_data(HOUSING_URL, HOUSING_PATH)

##############################################################################

## 데이터 읽기 함수

import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, 'housing.csv')
    return pd.read_csv(csv_path)

housing = load_housing_data()
#housing.head()

##############################################################################

## 테스트 세트 분리

import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

# income_cat 특성 생성
housing['income_cat'] = pd.cut(housing['median_income'],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing['income_cat']):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]
    
# income_cat 특성 삭제
for set_ in (strat_train_set, strat_test_set):
    set_.drop('income_cat', axis=1, inplace=True)
    
#print('strat_train_set size : {}'.format(len(strat_train_set)))
#print('strat_test_set size : {}'.format(len(strat_test_set)))

##############################################################################

## 훈련 세트 예측 변수와 레이블 분리

# drop()은 데이터 복사본을 만들며 strat_train_set에 영향을 주지 않음
housing = strat_train_set.drop('median_house_value', axis=1)
housing_labels = strat_train_set['median_house_value'].copy()

##############################################################################

## 변환 파이프 라인

# 특성 조합 변환기 정의
from sklearn.base import BaseEstimator, TransformerMixin

col_names = "total_rooms", "total_bedrooms", "population", "households"
rooms_ix, bedrooms_ix, population_ix, households_ix = [housing.columns.get_loc(c) for c in col_names]

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # *args 나 **kargs 가 아님
        self.add_bedrooms_per_room = add_bedrooms_per_room  
    def fit(self, X, y=None):
        return self # 더 이상 할 일이 없음
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix] # 가구당 방 갯수
        population_per_household = X[:, population_ix] / X[:, households_ix] # 가구당 인구수
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix] # 방 당 화장실 갯수
            return np.c_[X, 
                         rooms_per_household,
                         population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, 
                         rooms_per_household,
                         population_per_household]

# 수치형 특성 변환 파이프라인
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')), # 결측치 처리
    ('attribs_adder', CombinedAttributesAdder()), # 특성 조합
    ('std_scaler', StandardScaler()) # 수치형 특성 데이터 스케일링
])

# 통합 변환 파이프라인
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

housing_num = housing.drop('ocean_proximity', axis=1)
num_attribs = list(housing_num)
cat_attribs = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs) # 범주형 특성 데이터 숫자 변환
])

housing_prepared = full_pipeline.fit_transform(housing)
#housing_prepared

##############################################################################

## 모델 훈련

# 선형 회귀
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

# 의사결정나무 - 회귀
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

# 랜덤 포레스트 - 회귀
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)

# 서포트 벡터 머신
from sklearn.svm import SVR

svm_reg = SVR(kernel='linear')
svm_reg.fit(housing_prepared, housing_labels)

##############################################################################

## 평가 함수 정의

# RMSE
from sklearn.metrics import mean_squared_error
import numpy as np

def get_RMSE(model, X, y):
    predictions = model.predict(X)
    mse = mean_squared_error(y, predictions)
    rmse = np.sqrt(mse)
    return rmse

##############################################################################

## K-겹 교차 검증

from sklearn.model_selection import cross_val_score

def k_fold_cross_validation(model, X, y, cv=10):
    scores = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=cv)
    rmse_scores = np.sqrt(-scores)
    return rmse_scores

def display_scores(scores):
    print('점수:\n', scores)
    print('평균:\n', scores.mean())
    print('표준편차:\n', scores.std())

##############################################################################

## 전처리와 예측을 포함한 전체 파이프라인 및 모델 저장, 불러오기

# 전처리와 예측을 포함한 전체 파이프라인
full_pipeline_with_predictor = Pipeline([
    ('preparation', full_pipeline),
    ('linear', LinearRegression())
])

full_pipeline_with_predictor.fit(housing, housing_labels)

# 모델 저장
import joblib

my_model = full_pipeline_with_predictor
joblib.dump(my_model, '../models/my_model_ch02.pkl') # DIFF

# 모델 불러오기
my_model_loaded = joblib.load('../models/my_model_ch02.pkl')

##############################################################################

print('done')

done


In [2]:
print("linear regression's RMSE : ", get_RMSE(lin_reg, housing_prepared, housing_labels))
print("decision tree regression's RMSE : ", get_RMSE(tree_reg, housing_prepared, housing_labels))
print("randomforest regression's RMSE : ", get_RMSE(forest_reg, housing_prepared, housing_labels))
print("SVM regression's RMSE : ", get_RMSE(svm_reg, housing_prepared, housing_labels))

linear regression's RMSE :  68628.19819848923
decision tree regression's RMSE :  0.0
randomforest regression's RMSE :  18577.02783818094
SVM regression's RMSE :  111094.6308539982


In [3]:
lin_scores = k_fold_cross_validation(lin_reg, housing_prepared, housing_labels, 10)
print("linear regression's 10-fold cross validation scores:\n")
display_scores(lin_scores)

linear regression's 10-fold cross validation scores:

점수:
 [66782.73843989 66960.118071   70347.95244419 74739.57052552
 68031.13388938 71193.84183426 64969.63056405 68281.61137997
 71552.91566558 67665.10082067]
평균:
 69052.46136345084
표준편차:
 2731.6740017983466


In [4]:
forest_scores = k_fold_cross_validation(forest_reg, housing_prepared, housing_labels, 10)
print("randomForest regression's 10-fold cross validation scores:\n")
display_scores(forest_scores)

randomForest regression's 10-fold cross validation scores:

점수:
 [49480.6418009  47767.77535009 49829.45662081 52582.66482969
 49659.50496129 53573.43186104 48972.87872735 48053.32162021
 52920.87977829 50089.65089028]
평균:
 50293.02064399452
표준편차:
 1932.942172868786


<br>

## 2.7.1 그리드 탐색

- 튜닝의 가장 단순한 방법
  - 만족할 만한 하이퍼파라미터 조합을 찾을 때 까지 수동으로 하이퍼파라미터를 조정하는 것
  - 사이킷런의 `GridSearchCV`를 사용하는 것이 좋다.

<br>

### 2.7.1.1 `GridSearchCV` 을 통한 그리드 탐색

- 탐색하고자 하는 하이퍼파라미터와 시도해볼 값을 지정하기만 하면 된다.
- 그러면 가능한 모든 하이퍼파라미터 조합에 대해 교차 검증을 사용해 평가하게 된다.
- 어떤 하이퍼파라미터 값을 지정해야 할 지 모를 때는 **연속된 10의 거듭제곱 수**로 시도해보는 것이 좋다.
  - 더 세밀하게 탐색하려면 아래 예제의 `n_estimators` 하이퍼파라미터처럼 더 작은 값을 지정한다.  
  
  
- `GridSearchCV`가 (기본값인) `refit=True`로 초기화되었다면, 교차 검증으로 최적의 추정기를 찾은 다음 전체 훈련 세트로 다시 훈련 시킨다.
  - 일반적으로 데이터가 많을수록 성능이 향상되므로 좋은 방법이다.

- `RandomForestRegressor`에 대한 최적의 하이퍼파라미터 조합 탐색

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}
]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg,
                           param_grid,
                           cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
print('start')
grid_search.fit(housing_prepared, housing_labels)
print('done')

start
done


<br>

- `param_grid` 설정에 따라 사이킷런이 다음과 같은 과정을 수행한다.  
  
  
1. 첫 번째 dict에 있는 `n_estimators`와 `max_features` 하이퍼파라미터의 조합인 3x4=12개를 시도한다.
2. 두 번째 dict에 있는 하이퍼파라미터 조합인 2x3=6개를 시도한다.
  - 두 번째 dict에는 `bootstrap` 하이퍼파라미터를 `True`(기본값)가 아니라 `False`로 지정한다.
3. 모두 합하면 그리드 탐색이 `RandomForestRegressor` 하이퍼파라미터 값의 18(=12+6)개 조합을 탐색하고, 각각 다섯 번 모델을 훈련시킨다.
  - `cv=5` 이기 때문에 5-겹 교차 검증을 사용  
  
  
- 최종적으로 전체 훈련 횟수는 18x5=90 이 된다.

<br>

### 2.7.1.2 최적의 조합 확인

- 90번의 훈련을 시키는 것이 시간이 꽤 오래 걸리지만 다음과 같이 최적의 조합을 얻을 수 있다.

In [6]:
grid_search.best_params_

{'max_features': 8, 'n_estimators': 30}

- 최적의 조합의 값들이 탐색 범위의 최댓값이기 때문에 계속 점수가 향상될 가능성이 있다.
- 그러므로 더 큰 값으로 다시 검색해야 한다.

<br>

### 2.7.1.3 최적의 추정기 확인

In [7]:
grid_search.best_estimator_

RandomForestRegressor(max_features=8, n_estimators=30)

<br>

### 2.7.1.4 평가 점수 확인

In [8]:
cvres = grid_search.cv_results_
sorted(list(cvres.keys()))

['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'mean_train_score',
 'param_bootstrap',
 'param_max_features',
 'param_n_estimators',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split0_train_score',
 'split1_test_score',
 'split1_train_score',
 'split2_test_score',
 'split2_train_score',
 'split3_test_score',
 'split3_train_score',
 'split4_test_score',
 'split4_train_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score',
 'std_train_score']

In [9]:
pd.DataFrame(cvres)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_features,param_n_estimators,param_bootstrap,params,split0_test_score,split1_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.073377,0.013627,0.003799,0.000749,2,3,NaN,"{'max_features': 2, 'n_estimators': 3}",-3.989512e+09,-4.207301e+09,...,-4.188007e+09,1.663885e+08,18,-1.195363e+09,-1.081993e+09,-1.091434e+09,-1.145820e+09,-1.044374e+09,-1.111797e+09,5.289533e+07
1,0.214731,0.001470,0.009996,0.000002,2,10,NaN,"{'max_features': 2, 'n_estimators': 10}",-2.980508e+09,-3.082166e+09,...,-3.092153e+09,1.551577e+08,11,-6.113079e+08,-5.566262e+08,-5.719928e+08,-5.726385e+08,-5.884455e+08,-5.802022e+08,1.852536e+07
2,0.645390,0.004078,0.027592,0.000489,2,30,NaN,"{'max_features': 2, 'n_estimators': 30}",-2.636127e+09,-2.869020e+09,...,-2.783149e+09,1.263608e+08,8,-4.389239e+08,-4.263942e+08,-4.184283e+08,-4.320977e+08,-4.373466e+08,-4.306381e+08,7.521677e+06
3,0.099966,0.001265,0.003599,0.000800,4,3,NaN,"{'max_features': 4, 'n_estimators': 3}",-3.398790e+09,-3.527835e+09,...,-3.559546e+09,1.782250e+08,15,-9.709742e+08,-9.357760e+08,-1.004452e+09,-9.992613e+08,-8.712042e+08,-9.563335e+08,4.907573e+07
4,0.330292,0.003071,0.009998,0.000634,4,10,NaN,"{'max_features': 4, 'n_estimators': 10}",-2.721801e+09,-2.950951e+09,...,-2.821735e+09,1.236128e+08,9,-5.297499e+08,-5.310498e+08,-5.030901e+08,-5.342474e+08,-5.379263e+08,-5.272127e+08,1.238814e+07
5,0.980081,0.009517,0.026992,0.000893,4,30,NaN,"{'max_features': 4, 'n_estimators': 30}",-2.435992e+09,-2.558950e+09,...,-2.527962e+09,1.143343e+08,3,-3.958738e+08,-3.877814e+08,-3.775996e+08,-3.949214e+08,-3.842239e+08,-3.880800e+08,6.816813e+06
6,0.133356,0.002058,0.003799,0.000400,6,3,NaN,"{'max_features': 6, 'n_estimators': 3}",-3.304957e+09,-3.751900e+09,...,-3.485817e+09,2.098410e+08,14,-9.045114e+08,-9.434364e+08,-9.928847e+08,-9.336171e+08,-8.434333e+08,-9.235766e+08,4.916354e+07
7,0.447454,0.015951,0.010198,0.000982,6,10,NaN,"{'max_features': 6, 'n_estimators': 10}",-2.485392e+09,-2.784667e+09,...,-2.698163e+09,1.309921e+08,5,-5.219792e+08,-5.066329e+08,-4.892432e+08,-5.026123e+08,-5.084071e+08,-5.057749e+08,1.052180e+07
8,1.342961,0.011889,0.026793,0.001166,6,30,NaN,"{'max_features': 6, 'n_estimators': 30}",-2.344779e+09,-2.575154e+09,...,-2.512747e+09,1.441216e+08,2,-3.879460e+08,-3.897797e+08,-3.753737e+08,-3.959122e+08,-3.792738e+08,-3.856571e+08,7.401302e+06
9,0.173544,0.002869,0.003599,0.000490,8,3,NaN,"{'max_features': 8, 'n_estimators': 3}",-3.384410e+09,-3.339475e+09,...,-3.436456e+09,7.525340e+07,13,-9.642008e+08,-8.838035e+08,-8.561319e+08,-9.512181e+08,-8.732274e+08,-9.057163e+08,4.355492e+07


In [10]:
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

64714.81607081843 {'max_features': 2, 'n_estimators': 3}
55607.134618349985 {'max_features': 2, 'n_estimators': 10}
52755.56021730817 {'max_features': 2, 'n_estimators': 30}
59661.933325418104 {'max_features': 4, 'n_estimators': 3}
53120.00726861874 {'max_features': 4, 'n_estimators': 10}
50278.84703090894 {'max_features': 4, 'n_estimators': 30}
59040.80455263172 {'max_features': 6, 'n_estimators': 3}
51943.84925769594 {'max_features': 6, 'n_estimators': 10}
50127.312801814114 {'max_features': 6, 'n_estimators': 30}
58621.29627322658 {'max_features': 8, 'n_estimators': 3}
52011.78694035327 {'max_features': 8, 'n_estimators': 10}
50109.33117767366 {'max_features': 8, 'n_estimators': 30}
62455.05246945457 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
53799.10663549364 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
60769.36712106507 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
52232.597207813145 {'bootstrap': False, 'max_features': 3, 'n_estimators'

<br>

- 이 예에서는 `max_features=8`, `n_estimators=30` 일 때 최적의 솔루션이다.
- 이 때 RMSE 점수가 50,109로 앞서 기본 하이퍼파라미터 설정으로 얻은 50,293 보다 조금 더 좋다.

<br>

### 2.7.1.5 데이터 준비 단계에 그리드 탐색 적용

- 데이터 준비 단계를 하나의 하이퍼파라미터처럼 다룰 수 있다.
- ex) 그리드 탐색이 확실하지 않은 특성을 추가할 지 말지 자동으로 정할 수 있다.
  - `CombinedAttributedsAdder` 변환기의 `add_bedrooms_per_room` 하이퍼파라미터를 사용하여 특성을 추가할 지 결정
- 비슷하게 이상치나 값이 빈 특성을 다루거나 특성 선택 등을 자동으로 처리하는 데 그리드 탐색이 활용된다.
  - 데이터 준비 단계와 모델을 연결한 파이프라인을 그리드 탐색을 적용할 때 데이터 준비 단계를 캐싱하면 탐색 시간을 줄일 수 있다.
  - 참고 : [Pipeline에서 캐싱을 사용하기](https://goo.gl/cq9Nyb)